In [211]:
import os
import string,sys
from jinja2 import Environment, FileSystemLoader
import numpy as np

In [212]:
opt_f='bromide.log'
not_opt_f='coumarin.log'
with open(opt_f, 'r') as f:
    lines = f.readlines()

In [213]:
# Check if optimization was succesfull 
opt = False
for line in lines:
    if  "**** THE GEOMETRY SEARCH IS NOT CONVERGED! ****" in line:
        opt = True

In [236]:
# If optimization is successfull 
if not opt:
    data = [] 
    for num, line in enumerate(lines):
        if "TOTAL NUMBER OF ATOMS" in line:
            n_atoms = int(line.split()[5])
        if "EQUILIBRIUM" in line:
            start_coord_index = num + 4
            end_coord_index = start_coord_index + n_atoms
            for i in range(start_coord_index, end_coord_index):
                data.append(lines[i].split())
                
    # store opt coordinates into a list
    atomic_number = []
    element_symbol = []
    x_coord = []
    y_coord = []
    z_coord = []
    for line in data:
        element_symbol.append(line[0])
        atomic_number.append(line[1])
        x_coord.append(line[2])
        y_coord.append(line[3])
        z_coord.append(line[4])
    opt_coord = (element_symbol, x_coord, y_coord, z_coord)
    result = np.transpose(np.array(opt_coord))

In [237]:
# Option 1
if not opt:
    file_loader=FileSystemLoader("../templates")
    environment = Environment(loader=file_loader)
    template = environment.get_template("structure.xyz")

    MOLNAME="bromide"
    content = template.render(N_ATOMS=n_atoms, MOLNAME=MOLNAME, COORDINATES=result)
    print(content)

18 
bromide
['BR' '-0.0933869318' '2.8189374928' '0.3264436682']
['C' '0.0379065205' '0.9059296890' '0.1106576426']
['C' '1.2129668597' '0.3364498729' '-0.3630618823']
['C' '-1.0413632098' '0.0920420003' '0.4298464413']
['BR' '2.7191378817' '1.4585579884' '-0.8008101329']
['C' '1.2956054736' '-1.0396865624' '-0.5205134763']
['BR' '-2.6734335531' '0.8742509398' '1.0967697191']
['C' '-0.9391810721' '-1.2820173320' '0.2651366617']
['BR' '2.9471905083' '-1.8311739483' '-1.1635470565']
['C' '0.2235048153' '-1.8812853874' '-0.2213225169']
['BR' '-2.4441891907' '-2.4153207280' '0.7312369570']
['C' '0.3229579163' '-3.3842357617' '-0.4009576047']
['C' '0.8428428880' '-4.1012687552' '0.8560198076']
['H' '0.9730407574' '-3.6015073825' '-1.2328239009']
['H' '-0.6458840897' '-3.7763620898' '-0.6637045140']
['H' '1.8381483061' '-3.7647776866' '1.1165391949']
['H' '0.8862006147' '-5.1690993542' '0.6776776474']
['H' '0.1923355058' '-3.9302729952' '1.7041733447']



In [216]:
# Option 2    

# opt_xyz = open(MOLNAME+'.xyz','w')
# opt_xyz.write(str(n_atoms) +'\n')
# opt_xyz.write(MOLNAME +'\n')
if not opt:
        print(MOLNAME +'\n')
        print(str(n_atoms) +'\n')
        for i in range(n_atoms):
                line="{0:<4s}".format(element_symbol[i]) + ' ' 
                line+="{0:>16.10f}".format(float(x_coord[i])) + ' '
                line+="{0:>16.10f}".format(float(y_coord[i])) + ' '
                line+="{0:>16.10f}".format(float(z_coord[i])) + ' ' + '\n'
                print(line)
#         opt_xyz.write(line)
# opt_xyz.close()

bromide

18

BR      -0.0933869318     2.8189374928     0.3264436682 

BR      -0.0933869318     2.8189374928     0.3264436682 

C        0.0379065205     0.9059296890     0.1106576426 

C        0.0379065205     0.9059296890     0.1106576426 

C        1.2129668597     0.3364498729    -0.3630618823 

C        1.2129668597     0.3364498729    -0.3630618823 

C       -1.0413632098     0.0920420003     0.4298464413 

C       -1.0413632098     0.0920420003     0.4298464413 

BR       2.7191378817     1.4585579884    -0.8008101329 

BR       2.7191378817     1.4585579884    -0.8008101329 

C        1.2956054736    -1.0396865624    -0.5205134763 

C        1.2956054736    -1.0396865624    -0.5205134763 

BR      -2.6734335531     0.8742509398     1.0967697191 

BR      -2.6734335531     0.8742509398     1.0967697191 

C       -0.9391810721    -1.2820173320     0.2651366617 

C       -0.9391810721    -1.2820173320     0.2651366617 

BR       2.9471905083    -1.8311739483    -1.1635470565 

B

In [217]:
# If optimization is NOT successfull 

if  opt:
	file_loader=FileSystemLoader("../templates")
	environment = Environment(loader=file_loader)
	template = environment.get_template("gamess_input.inp")
	data = []
	for num, line in enumerate(lines):
		if "TOTAL NUMBER OF ATOMS" in line:
			n_atoms = int(line.split()[5])
		if "ALWAYS THE LAST POINT COMPUTED!" in line:
			start_coord_index = num + 4
			end_coord_index = start_coord_index + n_atoms
			for i in range(start_coord_index, end_coord_index):
				data.append(lines[i])

	SCFTYP='RHF'
	MULT=1 
	OPTTOL=0.0005
	NSTEP=100 
	GBASIS='N31' 
	NGAUSS=6
	MOLNAME="bromide"
	content = template.render(SCFTYP=SCFTYP, MULT=MULT, OPTTOL=OPTTOL,
			NSTEP=NSTEP, GBASIS=GBASIS, NGAUSS=NGAUSS, MOLNAME=MOLNAME, COORDINATES=data)
	print(content)